# Definitions

## Constants

In [ ]:
from typing import Dict,List

GATES_ACCESS_TYPE: Dict[str, List[str]] = {
    "h": ['U'],      # Hadamard
    "cx": ['R', 'U'], # Controlled-NOT: Read first (control), Update second (target)
    "r": ['U'],      # Single-qubit R gate: Update the qubit state
    "cr": ['R', 'U']  # Controlled-R gate: Read first (control), Update second (target)
}

FRONTIER_PRIORITY = ["h","cx","r"]

QISKIT_GATES = [
    # Single-qubit gates
    'h',       # Hadamard gate
    'x',       # Pauli-X gate
    'y',       # Pauli-Y gate
    'z',       # Pauli-Z gate
    's',       # S gate (pi/2 rotation over Z-axis)
    'sdg',     # S-dagger gate
    't',       # T gate (pi/4 rotation over Z-axis)
    'tdg',     # T-dagger gate
    'rx',      # Rotation around X-axis
    'ry',      # Rotation around Y-axis
    'rz',      # Rotation around Z-axis
    'p',       # Phase gate
    'u1',      # U1 gate (rotation around Z-axis)
    'u2',      # U2 gate (rotation around X+Z axis)
    'u3',      # U3 gate (generic single-qubit rotation)
    'id',      # Identity gate

    # Multi-qubit gates
    'cx',      # Controlled-NOT (CNOT) gate
    'cz',      # Controlled-Z gate
    'cy',      # Controlled-Y gate
    'ch',      # Controlled-H gate
    'swap',    # SWAP gate
    'ccx',     # Toffoli (CCX) gate
    'cswap',   # Controlled-SWAP (Fredkin) gate
    'cu1',     # Controlled-U1 gate
    'cu3',     # Controlled-U3 gate
    'crx',     # Controlled-RX gate
    'cry',     # Controlled-RY gate
    'crz',     # Controlled-RZ gate
    'rxx',     # Two-qubit XX rotation
    'ryy',     # Two-qubit YY rotation
    'rzz',     # Two-qubit ZZ rotation
    'xx_minus_yy',  # XX-YY gate

    # Measurement
    'measure', # Measurement operation

    # Barrier (not a gate, but often used in circuits)
    'barrier',

    # Reset operation
    'reset'
]

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Instruction
from typing import List, Tuple, Dict

class QuantumExtractor:
    def __init__(self, circuit: QuantumCircuit):
        self.circuit = circuit
        self.operations: List[Dict] = []
        self.instruction_sequence: Dict[str, List[List[int]]] = {}
        self.extract_info()
        self.group_instruction_sequences()

    def extract_info(self):
        for i, instruction_tuple in enumerate(self.circuit.data):
            instruction, qubits, _ = instruction_tuple
            op_info = {
                'index': i,
                'name': instruction.name,
                'qubits': [self.circuit.qubits.index(qubit) for qubit in qubits],
                'is_parameterized': isinstance(instruction, Instruction) and instruction.params,
                'label': instruction.label if hasattr(instruction, 'label') else None,
            }
            self.operations.append(op_info)

    def group_instruction_sequences(self):
        """
        Create a dictionary grouping all qubit sequences by unique labels, 
        preserving all redundancies.
        
        Returns:
        Dict[str, List[List[int]]]: A dictionary where keys are unique labels 
        and values are all qubit sequences for that label.
        """
        self.instruction_sequence = {}
        
        # Group all sequences by label
        for op in self.operations:
            if op['label'] is not None:
                # If label not in dictionary, initialize with empty list
                if op['label'] not in self.instruction_sequence:
                    self.instruction_sequence[op['label']] = []
                
                # Add the qubit sequence, including redundant ones
                self.instruction_sequence[op['label']].append(op['qubits'])
        
        return self.instruction_sequence

    def print_label_sequences(self):
        print("Label Sequences:")
        for label, sequences in self.instruction_sequence.items():
            print(f"{label}: {sequences}")
            
    def print_execution_sequence(self):
        print("Execution Sequence:")
        for op in self.operations:
            params = f" with params" if op['is_parameterized'] else ""
            print(f"Step {op['index']}: {op['name']} on qubit(s) {op['qubits']}{params} in inst : {op['label']}")


In [ ]:
import ast
from qiskit import QuantumCircuit
import astor
import json

class QiskitVisitor(ast.NodeTransformer):
    def __init__(self):
        self.current_loops: List[Tuple[int, int]] = []
        self.current_instrction_loop_map: List[Dict[str, str]] = []
        self.instrction_loop_map: Dict[str, List[Dict[str, str]]] = {}
        self.current_outter_loop_number = 0
        self.current_loop_depth = 0
        self.current_loop_number = 0
        self.current_inst_number = 0

    def visit_For(self, node):
        self.current_loop_number += 1
        self.current_loops.append((self.current_loop_number, self.current_loop_depth))
        self.current_loop_depth += 1
        
        # Visit children
        node = self.generic_visit(node)
        
        self.current_loop_depth -= 1
        self.current_loops.pop()
        if self.current_loop_depth == 0:
            self.instrction_loop_map[f"outter_loop_{self.current_outter_loop_number}"] =  self.current_instrction_loop_map
            self.current_outter_loop_number += 1
            self.current_instrction_loop_map = []
        
        return node

    def visit_Call(self, node):
        if isinstance(node.func, ast.Attribute) and node.func.attr in QISKIT_GATES:
            # Creating and adding the label
            label = f"S{self.current_inst_number}"
            label_kwarg = ast.keyword(arg='label', value=ast.Str(s=label))
            node.keywords.append(label_kwarg)
            # Append the corresponding loop informations
            self.current_instrction_loop_map.append({
                "gate": str((self.current_inst_number, node.func.attr)),
                "loops": str(self.current_loops),
                "label": label
            })
            self.current_inst_number += 1 
            return node
        
        # Remove the execute, run, and execute_on_backend function calls
        if isinstance(node.func, ast.Name) and node.func.id in ['execute', 'run', 'execute_on_backend']:
            return None
        
        return self.generic_visit(node)
    
    def visit_Assign(self, node):
        if isinstance(node.value, ast.Call) and isinstance(node.value.func, ast.Name) and node.value.func.id in ['execute', 'run', 'execute_on_backend']:
            # Skip the Assign node that contains the 'execute', 'run', or 'execute_on_backend' function calls
            return None
        else:
            return self.generic_visit(node)


    def transform(self, tree):
        return self.visit(tree)

In [ ]:
def get_label_loop_dimensions(instruction_loop_map):
    label_dimensions = {}
    for loop_id, outter_loop in instruction_loop_map.items():
        for mapping in outter_loop:
            label = mapping['label']
            loops = eval(mapping['loops'])  # Convert string representation to list
            
            if label not in label_dimensions:
                label_dimensions[label] = []
            
            label_dimensions[label].append(len(loops))
    
    return label_dimensions

In [ ]:
# REPLACE WITH ISL SET

def reconstruct_loops(sequence, num_loops):
    """
    Reconstruct the original loop boundaries and steps that generated the given sequence.
    
    Args:
    sequence (List[List[int]]): The sequence of loop iterations
    num_loops (int): Number of nested loops
    
    Returns:
    List[Dict]: A list of dictionaries, each representing a loop's boundaries and step
    """
    def is_valid_sequence(sequence):
        """Check if the sequence is valid (1D or 2D)"""
        if not sequence:
            return False
        return all(len(item) in [1, 2] for item in sequence)
    
    def detect_loop_boundaries(loop_dim):
        """
        Detect loop boundaries and step for a specific dimension
        
        Args:
        loop_dim (List[int]): List of values for a specific loop dimension
        
        Returns:
        Dict with start, end, and step of the loop or None for constant dimensions
        """
        # Remove duplicates while preserving order
        unique_values = []
        for val in loop_dim:
            if val not in unique_values:
                unique_values.append(val)
        
        # If no unique values or all values are the same, return special handling
        if len(unique_values) < 2:
            if len(set(loop_dim)) == 1:
                # Constant dimension
                return {
                    'start': unique_values[0],
                    'end': unique_values[0],
                    'step': 0,
                    'is_constant': True
                }
            return None
        
        # Detect step
        steps = [unique_values[i+1] - unique_values[i] for i in range(len(unique_values)-1)]
        
        # Check if all steps are consistent
        if len(set(steps)) != 1:
            return None
        
        step = steps[0]
        start = min(unique_values)
        end = max(unique_values)
        
        return {
            'start': start,
            'end': end,
            'step': step,
            'is_constant': False
        }
    
    # Validation
    if not is_valid_sequence(sequence):
        raise ValueError("Invalid sequence format. Must be a list of 1D or 2D integer lists.")
    
    # Ensure num_loops matches sequence dimension
    if num_loops != len(sequence[0]):
        raise ValueError(f"Number of loops ({num_loops}) does not match sequence dimension ({len(sequence[0])})")
    
    # Separate dimensions
    dimensions = [[] for _ in range(num_loops)]
    for item in sequence:
        for i, val in enumerate(item):
            dimensions[i].append(val)
    
    # Detect loop boundaries for each dimension
    loop_info = []
    for dim in dimensions:
        loop_boundary = detect_loop_boundaries(dim)
        if loop_boundary is None:
            raise ValueError(f"Could not determine loop boundaries for dimension {dim}")
        loop_info.append(loop_boundary)
    
    # Format output with more readable loop constraints
    formatted_loops = []
    for info in loop_info:
        if info['is_constant']:
            formatted_loop = {
                'constraint': f'i = {info["start"]}',
                'step': info['step']
            }
        else:
            formatted_loop = {
                'constraint': f"{info['start']} < i <= {info['end']}",
                'step': info['step']
            }
        formatted_loops.append(formatted_loop)
    
    return formatted_loops

# Execution Testing

In [ ]:
import astor

# Example qiskit code
qiskit_code = """    
from qiskit import QuantumCircuit
n = 3
qc = QuantumCircuit(n)
for i in range(n-1):
    qc.cx(i,n-1)
    for j in range(n-1):
        qc.cx(j,n-1)
job = execute(qc_1, simulator, shots=1024)
"""

# Creating the new AST and extract the loop->instruction map
tree = ast.parse(qiskit_code)
visitor = QiskitVisitor()
modified_tree = visitor.transform(tree)
instrction_loop_map = visitor.instrction_loop_map

# Translating the modified AST to code and Executing the code and extracting the Quantum register
modified_code = astor.to_source(modified_tree)
local_vars = {}
try : 
    exec(modified_code, globals(), local_vars)
except Exception as e:
    print(f"An error occurred: {e}")
qc = local_vars['qc']

# Extracting the gates sequences
extractor = QuantumExtractor(qc)
extractor.extract_info()

In [ ]:
print(qc)
print("-------------------------------------------------------------------------------------")
print(modified_code)
print("-------------------------------------------------------------------------------------")
for loop_id,outter_loop in instrction_loop_map.items():
    print(f"outter_loop_id : {loop_id}")
    for mapping in outter_loop:
        print(f"{json.dumps(mapping,indent=4)}")
print("-------------------------------------------------------------------------------------")
extractor.print_execution_sequence()
print("-------------------------------------------------------------------------------------")
extractor.print_label_sequences()
print("-------------------------------------------------------------------------------------")
loop_dimensions_mapping = get_label_loop_dimensions(instrction_loop_map)
print(loop_dimensions_mapping)

In [ ]:
sequence_2d = [[0, 2], [1, 2], [0, 3], [1, 3]]
print("2D Sequence Loops:")
print(reconstruct_loops(sequence_2d, 2))

# 1D sequence example
sequence_1d = [[0], [1], [2], [3]]
print("\n1D Sequence Loops:")
print(reconstruct_loops(sequence_1d, 1))

# Another 2D sequence with more complex pattern
sequence_2d_complex = [[0, 3], [1, 3], [2, 3], [0, 4], [1, 4], [2, 4]]
print("\nComplex 2D Sequence Loops:")
print(reconstruct_loops(sequence_2d_complex, 2))